In [ ]:
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import json
from llm import LLM, OpenAILLM, ClaudeLLM
import os

@dataclass
class ExecutionStep:
    """Execution step data class"""
    step_id: int
    description: str
    prompt: str
    result: Optional[str] = None
    check_result: Optional[bool] = None

class TaskDecomposition:
    def __init__(self, llm: LLM):
        self.llm: (OpenAILLM | ClaudeLLM) = llm
        self.conversation_history: List[Dict[str, str]] = []
        
    def _add_to_history(self, role: str, content: str):
        """Add conversation to history"""
        self.conversation_history.append({"role": role, "content": content})
        
    def generate_plan(self, task_description: str) -> List[ExecutionStep]:
        """Generate task execution plan"""
        planning_prompt = f"""
        Please analyze the following task carefully and generate a detailed execution plan.
        
        Task Description: {task_description}
        
        Return the execution plan in JSON format as follows:
        {{
            "steps": [
                {{
                    "step_id": 1,
                    "description": "Step description",
                    "prompt": "Specific prompt for executing this step"
                }}
            ]
        }}
        
        Requirements:
        1. Each step should be clear and specific
        2. Prompts should include necessary context and specific requirements
        3. Steps should be logically connected
        4. Ensure the final output matches the original task requirements
        5. Consider potential subtasks and dependencies
        
        The prompt for each step should be designed to:
        - Maintain context from previous steps
        - Focus on the specific objective of that step
        - Guide towards the overall task goal
        """
        
        self._add_to_history("user", planning_prompt)
        
        # Generate execution plan
        plan_json = self.llm.generate_json(
            prompt=planning_prompt,
            schema={"steps": list}
        )
        
        if not plan_json or "steps" not in plan_json:
            raise ValueError("Failed to generate plan")
            
        self._add_to_history("assistant", json.dumps(plan_json, ensure_ascii=False, indent=2))
        
        # Convert to ExecutionStep objects
        steps = []
        for step in plan_json["steps"]:
            steps.append(ExecutionStep(
                step_id=step["step_id"],
                description=step["description"],
                prompt=step["prompt"]
            ))
            
        return steps
        
    def execute_step(self, step: ExecutionStep, previous_results: List[str]) -> str:
        """Execute a single step"""
        context_prompt = f"""
        Current Task Progress:
        {self._format_previous_results(previous_results)}
        
        Current Step {step.step_id}: {step.description}
        
        Please execute the following task:
        {step.prompt}
        
        Requirements:
        1. Utilize results from previous steps effectively
        2. Ensure output meets the current step's objective
        3. Maintain alignment with original task goals
        4. Be specific and detailed in your execution
        5. Consider the impact on subsequent steps
        """
        
        self._add_to_history("user", context_prompt)
        
        result = self.llm.generate_text(
            prompt=context_prompt,
            max_tokens=2000
        )
        
        self._add_to_history("assistant", result)
        
        return result
        
    def check_step_result(self, step: ExecutionStep, result: str, task_description: str) -> bool:
        """Check step execution result"""
        check_prompt = f"""
        Please evaluate if the following execution result meets the requirements:
        
        Original Task: {task_description}
        Step Description: {step.description}
        Execution Result: {result}
        
        Evaluate based on:
        1. Alignment with step objectives
        2. Consistency with original task
        3. Quality and completeness
        4. Logical connection with other steps
        
        Return evaluation result in JSON format:
        {{
            "passed": true/false,
            "reason": "Detailed explanation of the evaluation",
            "suggestions": "Improvement suggestions if any"
        }}
        """
        
        self._add_to_history("user", check_prompt)
        
        check_result = self.llm.generate_json(
            prompt=check_prompt,
            schema={"passed": bool, "reason": str, "suggestions": str}
        )
        
        self._add_to_history("assistant", json.dumps(check_result, ensure_ascii=False, indent=2))
        
        return check_result.get("passed", False)
    
    def _format_previous_results(self, results: List[str]) -> str:
        """Format previous execution results"""
        if not results:
            return "No previous execution results"
            
        formatted = "Previous Step Results:\n"
        for i, result in enumerate(results):
            formatted += f"\nStep {i+1} Result:\n{result}\n"
        return formatted
        
    def execute_task(self, task_description: str, enable_checking: bool = True) -> Dict[str, Any]:
        """Execute complete task"""
        self.conversation_history = []
        
        # 1. Generate execution plan
        steps = self.generate_plan(task_description)
        
        # 2. Execute steps sequentially
        results = []
        for step in steps:
            # Execute step
            result = self.execute_step(step, results)
            step.result = result
            
            # Check result if enabled
            if enable_checking:
                passed = self.check_step_result(step, result, task_description)
                step.check_result = passed
                
                # Retry if check failed
                if not passed:
                    retry_prompt = f"""
                    The previous execution of step {step.step_id} did not meet requirements.
                    
                    Original Task: {task_description}
                    Step Description: {step.description}
                    Previous Result: {result}
                    
                    Please re-execute this step with improved focus on:
                    1. Addressing the identified issues
                    2. Maintaining alignment with task goals
                    3. Ensuring quality and completeness
                    """
                    
                    self._add_to_history("user", retry_prompt)
                    result = self.llm.generate_text(
                        prompt=retry_prompt,
                        max_tokens=2000
                    )
                    step.result = result
            
            results.append(result)
        
        # 3. Integrate final result
        final_integration_prompt = f"""
        Please integrate all step results into a final output:
        {self._format_previous_results(results)}
        
        Requirements:
        1. Ensure the final output fully meets the original task requirements: {task_description}
        2. Maintain coherence and completeness
        3. Address all aspects of the original task
        4. Ensure logical flow and consistency
        5. Polish and refine the integrated result
        
        Focus on:
        - Clear and cohesive presentation
        - Logical connection between components
        - Comprehensive coverage of all requirements
        - Professional and refined output
        """
        
        self._add_to_history("user", final_integration_prompt)
        final_result = self.llm.generate_text(
            prompt=final_integration_prompt,
            max_tokens=2000
        )
        self._add_to_history("assistant", final_result)
        
        # 4. Return complete execution record
        return {
            "final_result": final_result,
            "steps": [
                {
                    "step_id": step.step_id,
                    "description": step.description,
                    "result": step.result,
                    "check_result": step.check_result
                }
                for step in steps
            ],
            "conversation_history": self.conversation_history
        }

# Usage example
def example_usage():
    llm = LLM(api_key=os.environ.get('GPT_API_KEY'), verbose=True)
    decomposer = TaskDecomposition(llm)
    
    # Example task: Rewrite text in different style
    task = """
请列举出杰克伦敦的黄祸的一个片段，随后将其改写成鲁迅的文风（中文）
    """
    
    result = decomposer.execute_task(task, enable_checking=True)
    
    print("Final Result:", result["final_result"])
    print("\nExecution Steps:")
    for step in result["steps"]:
        print(f"\nStep {step['step_id']}:")
        print(f"Description: {step['description']}")
        print(f"Result: {step['result']}")
        if step['check_result'] is not None:
            print(f"Check Result: {'Passed' if step['check_result'] else 'Failed'}")

In [5]:
example_usage()

{
    "steps": [
        {
            "step_id": 1,
            "description": "Identify a relevant excerpt from Jack London's 'The Yellow Peril'.",
            "prompt": "Read 'The Yellow Peril' by Jack London and select a significant excerpt that captures the essence of the story. Ensure the excerpt is concise and representative of the themes present in the work."
        },
        {
            "step_id": 2,
            "description": "Analyze the selected excerpt for key themes and stylistic elements.",
            "prompt": "Examine the chosen excerpt for its key themes, tone, and stylistic elements. Take notes on the language, imagery, and emotional impact to understand how to effectively translate it into Lu Xun's style."
        },
        {
            "step_id": 3,
            "description": "Research Lu Xun's writing style and themes.",
            "prompt": "Study Lu Xun's works to identify his unique writing style, including his use of language, themes, and narrative tec